In [1]:
import redis

In [2]:
db = redis.Redis(host='localhost', port=6379, db=13)

In [3]:
keys = db.keys()

In [4]:
len(keys)

51977

In [5]:
import json

In [6]:
keys[0]

b'ekotip'

In [7]:
db['agung']

b'{"definisi": ["[a.gung]  |  \\u0627\\u0762\\u0648\\u06a0Definisi : 1. (paling) besar, terutama: balai ~; layar ~; tiang ~; 2. yg tertinggi: pemerintah ~; 3. teristimewa, terkemuka, terutama: Allahyarham Tun Abdul Razak seorang pemimpin yg ~; Allahyarham P. Ramlee seniman ~ Malaysia; tamu ~; 4. yg terbuka kpd dan utk dihadiri oleh semua ahli (bkn mesyuarat, persidangan, dsb): persidangan ~ UMNO; mesyuarat ~ Kesatuan Sekerja DBP; mengagungkan menjadikan (menganggap dsb) agung, membesarkan, memuliakan, me\\u00adninggikan: al-Quran ~ ilmu pengetahuan dan memuliakan ahli ilmu; mengagung-agungkan selalu (sering kali) mengagungkan, sangat mengagungkan; keagungan kebesaran, kemuliaan, keting\\u00adgian: ~ Abdullah sbg pengarang terletak pd konsepsinya yg moden; keagungan kebesaran, kemuliaan, keting\\u00adgian: ~ Abdullah sbg pengarang terletak pd konsepsinya yg moden; pengagungan perbuatan dsb mengagungkan: antara yg menghalang perkembangan pesat bahasa Melayu ialah kekurangan bahan rujuk\\

In [8]:
d = json.loads(db[keys[0]])

In [9]:
from tqdm import tqdm

selected = {}

for k in tqdm(keys):
    d = json.loads(db[k])
    if not len(d['definisi']):
        continue
        
    selected[k.decode()] = d

100%|██████████████████████████████████████████████████████████████████████████████████| 51977/51977 [00:01<00:00, 29035.20it/s]


In [10]:
len(json.loads(db['agung'])['definisi'])

3

In [11]:
len(selected)

31665

In [12]:
import re
from unidecode import unidecode

alphabet = 'qwertyuiopasdfghjklzxcvbnm'
consonants = 'bcdfghjklmnpqrstvwxyz'
vowels = 'aeiou'
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

In [13]:
import enchant
d = enchant.Dict('en_US')

In [14]:
results = set()

for data in tqdm(selected.values()):
    
    texts = []
    for definisi in data['definisi']:
        texts.extend(definisi.split('Definisi : ')[1].split())

    if data['tesaurus']:
        texts.extend(data['tesaurus'])
    
    for k in range(len(texts)):
        
        t = texts[k]
        t = unidecode(t)
        t = t.replace('--', '-')
        for p in PUNCTUATION:
            t = t.replace(p, '')

        if all([c in consonants for c in t]):
            continue

        if len(t) <= 3:
            continue

        if len(re.findall(r'\d+', t)):
            continue

        if t[0] in '-':
            continue

        if t in ['yg', 'utk']:
            continue
            
        if t[-1] in '-':
            continue
        
        if 'dll' in t:
            continue
            
        t = t.lower()
        
        if d.check(t):
            continue
            
        if t in results:
            continue
            
        results.add(t)

100%|███████████████████████████████████████████████████████████████████████████████████| 31665/31665 [00:10<00:00, 2939.17it/s]


In [15]:
len(results)

51655

In [16]:
results

{'basmati',
 'perenggannya',
 'tohok-menohok',
 'poket',
 'perencat',
 'rangkung',
 'panitera',
 'krep',
 'bisulnya',
 'menampek',
 'berpetak-petak',
 'dibesarkan',
 'diperlenteh',
 'mengibaskan',
 'panduan',
 'hafaz',
 'stercorea',
 'honggek',
 'menjahit',
 'politena',
 'kelitik',
 'menggembirakan',
 'pembungkusnya',
 'bermalai',
 'keteraturan',
 'berdendam',
 'seni',
 'campuran',
 'bertepek-tepek',
 'kekampungan',
 'hidung',
 'ber-intikan',
 'kiai',
 'berburuk-buruk',
 'punggahan',
 'rerot',
 'penduduk-nya',
 'tenis',
 'memengat',
 'bergari',
 'berkejap-kejap',
 'menadah',
 'tekanannya',
 'berdayung',
 'berumpan',
 'ber-tolong',
 'minum-minum',
 'airlah',
 'mencerca',
 'pinser',
 'berpantis',
 'serobok',
 'kedengaranlah',
 'dicam',
 'tercelup',
 'celupan',
 'nitrit',
 'terpencilketerasingan',
 'makalah',
 'petugas-petugas',
 'selang-menyelang',
 'ulam-ulam',
 'kalaupun',
 'tercinta',
 'serbuan',
 'meronggok',
 'gratissima',
 'alami',
 'syarah',
 'berkalang',
 'solaris',
 'mem-bantah'

In [17]:
with open('dbp-v5.json', 'w') as fopen:
    json.dump(selected, fopen)

In [18]:
with open('malay-words-v5.json', 'w') as fopen:
    json.dump(list(results), fopen)